##### Packages

In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler
from scipy import stats

#### Load Data

In [2]:
# 年增率(YoY) (percentage change)
def calculate_YoY(df0, df1, column_name):
    df1[f'{column_name}_YoY'] = (df1[column_name] - df0[column_name]) / df0[column_name] * 100

In [3]:
fileINFO = "database/上市公司基本資料.xlsx"
dfINFO = pd.read_excel(fileINFO)

##### 上市_溫室氣體排放

In [4]:
# Load data
file110_gas = "database/110_ESG資訊接露_上市_溫室氣體排放.csv"
file111_gas = "database/111_ESG資訊接露_上市_溫室氣體排放.csv"
df110_gas = pd.read_csv(file110_gas)
df111_gas = pd.read_csv(file111_gas)
df110_gas.fillna(value=0, inplace=True)
df111_gas.fillna(value=0, inplace=True)
if not df111_gas.columns.all() == df110_gas.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()

df111_gas[['scope1_ton_CO2e', 'scope2_ton_CO2e', 'scope3_ton_CO2e']] = df111_gas[['scope1_ton_CO2e', 'scope2_ton_CO2e', 'scope3_ton_CO2e']].astype(int)
df110_gas[['scope1_ton_CO2e', 'scope2_ton_CO2e', 'scope3_ton_CO2e']] = df110_gas[['scope1_ton_CO2e', 'scope2_ton_CO2e', 'scope3_ton_CO2e']].astype(int)

# 計算範疇1,2,3排放量年增率(YoY) (percentage change)
calculate_YoY(df110_gas, df111_gas, 'scope1_ton_CO2e')
calculate_YoY(df110_gas, df111_gas, 'scope2_ton_CO2e')
calculate_YoY(df110_gas, df111_gas, 'scope3_ton_CO2e')

# 計算總排放量(範疇1+2)年增率(YoY) (percentage change)
df111_gas["scope12_ton_CO2e_YoY"] = (df111_gas["scope1_ton_CO2e"] + df111_gas["scope2_ton_CO2e"] - (df110_gas["scope1_ton_CO2e"] + df110_gas["scope2_ton_CO2e"])) / (df110_gas["scope1_ton_CO2e"] + df110_gas["scope2_ton_CO2e"]) * 100

# Select desired columns for the new dataframe
desired_columns = ['id', 'scope1_ton_CO2e_YoY', 'scope2_ton_CO2e_YoY', 'scope3_ton_CO2e_YoY', 'scope12_ton_CO2e_YoY']
df_gas = df111_gas[desired_columns]

df_all = df_gas.copy()

##### 上市_能源管理

In [5]:
# Load data
file110_energy = "database/110_ESG資訊接露_上市_能源管理.csv"
file111_energy = "database/111_ESG資訊接露_上市_能源管理.csv"
df110_energy = pd.read_csv(file110_energy)
df111_energy = pd.read_csv(file111_energy)
df110_energy.fillna(value=0, inplace=True)
df111_energy.fillna(value=0, inplace=True)
if not df111_energy.columns.all() == df110_energy.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()

df111_energy[['reenergy_usage_rate']] = df111_energy[['reenergy_usage_rate']].astype(float)
df110_energy[['reenergy_usage_rate']] = df110_energy[['reenergy_usage_rate']].astype(float)

# 計算再生能源使用率年增率(YoY) (percentage change)
calculate_YoY(df110_energy, df111_energy, 'reenergy_usage_rate')

# 是否有通過能源管理認證(ISO 50001)
iso50001 = '50001'
df111_energy['ISO50001'] = df111_energy['energy_ISO'].str.contains(iso50001)
df110_energy['ISO50001'] = df110_energy['energy_ISO'].str.contains(iso50001)

merged_df = pd.merge(df111_energy, df110_energy, on='id', suffixes=('_df111', '_df110'), how='inner')

for index, row in merged_df.iterrows():
    iso50001_df111 = row['ISO50001_df111']
    iso50001_df110 = row['ISO50001_df110']
    
    if iso50001_df111 == iso50001_df110 and iso50001_df111 == True:
        df111_energy.loc[index, 'ISO50001'] = 3 # T/T
    elif iso50001_df111 and not iso50001_df110:
        df111_energy.loc[index, 'ISO50001'] = 4 # F/T
    elif not iso50001_df111 and not iso50001_df110:
        df111_energy.loc[index, 'ISO50001'] = 1 # T/F
    else:
        df111_energy.loc[index, 'ISO50001'] = 2
        
# Select desired columns for the new dataframe
desired_columns = ["id", 'reenergy_usage_rate_YoY', 'ISO50001']
df_energy = df111_energy[desired_columns]

df_all = df_all.merge(df_energy, on='id', how='outer')

In [6]:
df111_energy

,id,name,reenergy_usage_rate,energy_boundary,energy_ISO,reenergy_usage_rate_YoY,ISO50001
0,1101,�x�d,0.00006,�x�d�x�W���餽�q (�[�\��B�����`���B��s�ǡB2�y���d...,ISO 50001,-40.0,3
1,1102,�Ȫd,0.00000,�Ȭw���d���餽�q,ISO 50001,NaN,3
2,1103,�Ūd,0.00000,�Ūd�x�W���餽�q,ISO 50001,NaN,3
3,1104,���d,0.00000,�����q,N,NaN,1
4,1108,����,0.00000,0,N,NaN,1
...,...,...,...,...,...,...,...
975,9944,�s�R,0.00000,0,N,NaN,False
976,9945,�����s,0.00000,0,N,NaN,False
977,9946,�T�o�a��,0.00000,0,N,NaN,False
978,9955,���s,0.00000,0,N,NaN,False


##### 上市_水資源管理

In [7]:
# Load data
file110_water = "database/110_ESG資訊接露_上市_水資源管理.csv"
file111_water = "database/111_ESG資訊接露_上市_水資源管理.csv"
df110_water = pd.read_csv(file110_water)
df111_water = pd.read_csv(file111_water)
df110_water.fillna(value=0, inplace=True)
df111_water.fillna(value=0, inplace=True)

if not df111_water.columns.all() == df110_water.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()

df111_water[['water_usage_ton']] = df111_water[['water_usage_ton']].astype(float)
df110_water[['water_usage_ton']] = df110_water[['water_usage_ton']].astype(float)

# 計算用水量年增率(YoY) (percentage change)
calculate_YoY(df110_water, df111_water, 'water_usage_ton')

# Select desired columns for the new dataframe
desired_columns = ["id", 'water_usage_ton_YoY']
df_water = df111_water[desired_columns]

df_all = df_all.merge(df_water, on='id', how='outer')

##### 上市_廢棄物管理

In [8]:
# Load data
file110_trash = "database/110_ESG資訊接露_上市_廢棄物管理.csv"
file111_trash = "database/111_ESG資訊接露_上市_廢棄物管理.csv"
df110_trash = pd.read_csv(file110_trash)
df111_trash = pd.read_csv(file111_trash)
df110_trash.fillna(value=0, inplace=True)
df111_trash.fillna(value=0, inplace=True)

if not df111_trash.columns.all() == df110_trash.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()

df111_trash[['harmful_trash_ton', 'harmfless_trash_ton', 'trash_ton', 'intensity_trash']] = df111_trash[['harmful_trash_ton', 'harmless_trash_ton', 'trash_ton', 'intensity_trash']].astype(float)
df110_trash[['harmful_trash_ton', 'harmless_trash_ton', 'trash_ton', 'intensity_trash']] = df110_trash[['harmful_trash_ton', 'harmless_trash_ton', 'trash_ton', 'intensity_trash']].astype(float)

# 計算廢棄物年增率(YoY) (percentage change)
calculate_YoY(df110_trash, df111_trash, 'harmful_trash_ton') # 有害廢棄物(噸)
calculate_YoY(df110_trash, df111_trash, 'harmless_trash_ton') # 無害廢棄物(噸)
calculate_YoY(df110_trash, df111_trash, 'trash_ton') # 有+無害廢棄物(噸)
calculate_YoY(df110_trash, df111_trash, 'intensity_trash') # 廢棄物密集度

# 是否有通過環保認證(ISO 14001)
iso14001 = '14001'
df111_trash['ISO14001'] = df111_trash['trash_ISO'].str.contains(iso14001)
df110_trash['ISO14001'] = df110_trash['trash_ISO'].str.contains(iso14001)

merged_df = pd.merge(df111_trash, df110_trash, on='id', suffixes=('_df111', '_df110'), how='inner')

for index, row in merged_df.iterrows():
    iso14001_df111 = row['ISO14001_df111']
    iso14001_df110 = row['ISO14001_df110']
    
    if iso14001_df111 and iso14001_df110:
        df111_trash.at[index, 'ISO14001'] = 3 # T/T
    elif iso14001_df111 and not iso14001_df110:
        df111_trash.at[index, 'ISO14001'] = 4 # F/T
    elif not iso14001_df111 and not iso14001_df110:
        df111_trash.at[index, 'ISO14001'] = 1 # T/F
    else:
        df111_trash.at[index, 'ISO14001'] = 2
        
# Select desired columns for the new dataframe
desired_columns = ["id", 'harmful_trash_ton_YoY', 'harmless_trash_ton_YoY', 'trash_ton_YoY', 'intensity_trash_YoY', 'ISO14001']
df_trash = df111_trash[desired_columns]

df_all = df_all.merge(df_trash, on='id', how='outer')

##### 上市_人力發展

In [9]:
# Load data
file110_human = "database/110_ESG資訊接露_上市_人力發展.csv"
file111_human = "database/111_ESG資訊接露_上市_人力發展.csv"
df110_human = pd.read_csv(file110_human)
df111_human = pd.read_csv(file111_human)
df110_human.fillna(value=0, inplace=True)
df111_human.fillna(value=0, inplace=True)

if not df111_human.columns.all() == df110_human.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()
# welfare_mean: 平均員工福利費用
# ft_salary_mean: 非主管職之全時員工平均薪資
# pt_fe_ratio: 主管職女性比例
# oa_num: 職災人數
# oa_ratio: 職災比例
df111_human[['welfare_mean','salary_mean','ft_salary_mean','ft_salary_med','oa_num']] = df111_human[['welfare_mean','salary_mean','ft_salary_mean','ft_salary_med','oa_num']].astype(int)
df110_human[['welfare_mean','salary_mean','ft_salary_mean','ft_salary_med','oa_num']] = df110_human[['welfare_mean','salary_mean','ft_salary_mean','ft_salary_med','oa_num']].astype(int)
df111_human[['pt_fe_ratio','oa_ratio']] = df111_human[['pt_fe_ratio','oa_ratio']].astype(float)
df110_human[['pt_fe_ratio','oa_ratio']] = df110_human[['pt_fe_ratio','oa_ratio']].astype(float)

# 計算年增率(YoY) (percentage change)
calculate_YoY(df110_human, df111_human, 'welfare_mean') # 平均員工福利費用
calculate_YoY(df110_human, df111_human, 'salary_mean')
calculate_YoY(df110_human, df111_human, 'ft_salary_mean') 
calculate_YoY(df110_human, df111_human, 'ft_salary_med')  
calculate_YoY(df110_human, df111_human, 'pt_fe_ratio') # 主管職女性比例
calculate_YoY(df110_human, df111_human, 'oa_num') # 職災人數
calculate_YoY(df110_human, df111_human, 'oa_ratio') # 職災比例

# Select desired columns for the new dataframe
desired_columns = ["id", 'welfare_mean_YoY', 'salary_mean_YoY', 'ft_salary_mean_YoY', 'ft_salary_med_YoY', 'oa_num_YoY', 'oa_ratio_YoY', 'pt_fe_ratio_YoY']
df_human = df111_human[desired_columns]

df_all = df_all.merge(df_human, on='id', how='outer')

##### 上市_董事會

In [10]:
# Load data
file110_bod = "database/110_ESG資訊接露_上市_董事會.csv"
file111_bod = "database/111_ESG資訊接露_上市_董事會.csv"
df110_bod = pd.read_csv(file110_bod)
df111_bod = pd.read_csv(file111_bod)
df110_bod.fillna(value=0, inplace=True)
df111_bod.fillna(value=0, inplace=True)

if not df111_bod.columns.all() == df110_bod.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()

# fe_d_ratio: Female Derictor 女性董事席次及比率-比率
# da_rate: Director Attendance 董事出席董事會出席率
# dfe_ratio: Derictor's Further Education 董監事進修時數符合進修要點比率

df111_bod[['d_seats','id_seats','fe_d_seats']] = df111_bod[['d_seats','id_seats','fe_d_seats']].astype(int)
df110_bod[['d_seats','id_seats','fe_d_seats']] = df110_bod[['d_seats','id_seats','fe_d_seats']].astype(int)
df111_bod[['fe_d_ratio','da_rate','dfe_ratio']] = df111_bod[['fe_d_ratio','da_rate','dfe_ratio']].astype(float)
df110_bod[['fe_d_ratio','da_rate','dfe_ratio']] = df110_bod[['fe_d_ratio','da_rate','dfe_ratio']].astype(float)
df110_bod['id_ratio'] = (df110_bod['id_seats'] / df110_bod['d_seats']) * 100
df111_bod['id_ratio'] = (df111_bod['id_seats'] / df111_bod['d_seats']) * 100

# 計算年增率(YoY) (percentage change)
calculate_YoY(df110_bod, df111_bod, 'd_seats') # 董事席次(含獨立董事)(席)
calculate_YoY(df110_bod, df111_bod, 'id_seats') # 獨立董事席次(席)
calculate_YoY(df110_bod, df111_bod, 'id_ratio') # 獨立董事比率
calculate_YoY(df110_bod, df111_bod, 'fe_d_seats') # 女性董事席次及比率-比率
calculate_YoY(df110_bod, df111_bod, 'da_rate') # 董事出席董事會出席率
calculate_YoY(df110_bod, df111_bod, 'dfe_ratio') # 董監事進修時數符合進修要點比率

# Select desired columns for the new dataframe
desired_columns = ["id", 'd_seats_YoY', 'id_seats_YoY', 'id_ratio_YoY', 'fe_d_seats_YoY', 'da_rate_YoY', 'dfe_ratio_YoY']
df_bod = df111_bod[desired_columns]

df_all = df_all.merge(df_bod, on='id', how='outer')

##### 上市_投資人溝通

In [11]:
# Load data
file110_investor = "database/110_ESG資訊接露_上市_投資人溝通.csv"
file111_investor = "database/111_ESG資訊接露_上市_投資人溝通.csv"
df110_investor = pd.read_csv(file110_investor)
df111_investor = pd.read_csv(file111_investor)
df110_investor.fillna(value=0, inplace=True)
df111_investor.fillna(value=0, inplace=True)
if not df111_investor.columns.all() == df110_investor.columns.all():
    print("Error: 欄位名稱不完全相符")
    exit()

# call_num: 公司年度召開法說會次數(次)
df111_investor[['call_num']] = df111_investor[['call_num']].astype(int)
df110_investor[['call_num']] = df110_investor[['call_num']].astype(int)

# 計算召開法人說明會次數年增率(YoY) (percentage change)
calculate_YoY(df110_investor, df111_investor, 'call_num')

# Select desired columns for the new dataframe
desired_columns = ['id', 'call_num_YoY']
df_investor = df111_investor[desired_columns]

df_all = df_all.merge(df_investor, on='id', how='outer')

##### 上市_員工福利政策及權益維護措施(爬蟲)

In [12]:
# Load data
file110_policy = "database/110_policyInfo.csv"
file111_policy = "database/111_policyInfo.csv"
df110_policy = pd.read_csv(file110_policy)
df111_policy = pd.read_csv(file111_policy)

def policy_score_rating(df0, df1, col_name):
    merged_df = pd.merge(df0, df1, on='代號', suffixes=('_df111', '_df110'), how='inner')

    for index, row in merged_df.iterrows():
        col_name_df111 = row[f'{col_name}_df111']
        col_name_df110 = row[f'{col_name}_df110']

        if col_name_df111 and col_name_df110:
            df1.at[index, f'{col_name}_score'] = 3  # 1/1
        elif col_name_df111 and not col_name_df110:
            df1.at[index, f'{col_name}_score'] = 4  # 1/0
        elif not col_name_df111 and not col_name_df110:
            df1.at[index, f'{col_name}_score'] = 1  # 0/0
        else:
            df1.at[index, f'{col_name}_score'] = 2  # 0/1

In [13]:
policy_score_rating(df110_policy, df111_policy, col_name='free_speech')
policy_score_rating(df110_policy, df111_policy, col_name='staff_train')
policy_score_rating(df110_policy, df111_policy, col_name='occupation_health')
policy_score_rating(df110_policy, df111_policy, col_name='consumer_neg')
policy_score_rating(df110_policy, df111_policy, col_name='supplier_esg')
policy_score_rating(df110_policy, df111_policy, col_name='risk_management')
policy_score_rating(df110_policy, df111_policy, col_name='social')
policy_score_rating(df110_policy, df111_policy, col_name='conservation')
policy_score_rating(df110_policy, df111_policy, col_name='nocorrupt')
policy_score_rating(df110_policy, df111_policy, col_name='salary')

df_all = df_all.rename(columns={"id": "代號"})

desired_columns = ['代號', 'free_speech_score', 'staff_train_score', 'occupation_health_score', 
                   'conservation_score', 'consumer_neg_score', 'supplier_esg_score', 
                   'risk_management_score', 'social_score', 'nocorrupt_score', 'salary_score']
df_policy = df111_policy[desired_columns]

df_all = df_all.merge(df_policy, on='代號', how='outer')

KeyError: 'salary_df111'

#### 可以拿來分析的資料檔

##### 產業代碼對應

In [ ]:
id_columns = ['代號', '主計處產業名']
dfINFO = dfINFO[id_columns]

df = df_all.merge(dfINFO, on='代號', how='outer')
df = df.dropna(subset=['主計處產業名'])

In [ ]:
df.loc[df["主計處產業名"].str[0:2] == "08", "主計處產業名"] = 1
df.loc[df["主計處產業名"].str[0:2] == "09", "主計處產業名"] = 2
df.loc[df["主計處產業名"].str[0:2] == "10", "主計處產業名"] = 2
df.loc[df["主計處產業名"].str[0:2] == "11", "主計處產業名"] = 3
df.loc[df["主計處產業名"].str[0:2] == "12", "主計處產業名"] = 4
df.loc[df["主計處產業名"].str[0:2] == "13", "主計處產業名"] = 5
df.loc[df["主計處產業名"].str[0:2] == "14", "主計處產業名"] = 6
df.loc[df["主計處產業名"].str[0:2] == "15", "主計處產業名"] = 7
df.loc[df["主計處產業名"].str[0:2] == "16", "主計處產業名"] = 8
df.loc[df["主計處產業名"].str[0:2] == "17", "主計處產業名"] = 9
df.loc[df["主計處產業名"].str[0:2] == "18", "主計處產業名"] = 10
df.loc[df["主計處產業名"].str[0:2] == "19", "主計處產業名"] = 11
df.loc[df["主計處產業名"].str[0:2] == "20", "主計處產業名"] = 12
df.loc[df["主計處產業名"].str[0:2] == "21", "主計處產業名"] = 13
df.loc[df["主計處產業名"].str[0:2] == "22", "主計處產業名"] = 15
df.loc[df["主計處產業名"].str[0:2] == "23", "主計處產業名"] = 14
df.loc[df["主計處產業名"].str[0:2] == "24", "主計處產業名"] = 16
df.loc[df["主計處產業名"].str[0:2] == "25", "主計處產業名"] = 17
df.loc[df["主計處產業名"].str[0:2] == "26", "主計處產業名"] = 18
df.loc[df["主計處產業名"].str[0:2] == "27", "主計處產業名"] = 19
df.loc[df["主計處產業名"].str[0:2] == "28", "主計處產業名"] = 20
df.loc[df["主計處產業名"].str[0:2] == "29", "主計處產業名"] = 21
df.loc[df["主計處產業名"].str[0:2] == "30", "主計處產業名"] = 22
df.loc[df["主計處產業名"].str[0:2] == "31", "主計處產業名"] = 23
df.loc[df["主計處產業名"].str[0:2] == "32", "主計處產業名"] = 24
df.loc[df["主計處產業名"].str[0:2] == "33", "主計處產業名"] = 25
df.loc[df["主計處產業名"].str[0:2] == "34", "主計處產業名"] = 26

df.rename(columns={'主計處產業名': 'industry_id'}, inplace=True)

mask = df['industry_id'].str.len() >= 3
df = df[~mask]

In [ ]:
# 變成excel檔
df.to_csv("database/ESGinfo_TimeSeries.csv", index=False, encoding='big5')